In [1]:
import scvelo as scv
import pandas as pd
import numpy as np
import scanpy as sc
import glob
import os

ModuleNotFoundError: No module named 'scvelo'

In [ ]:
sample = 'cfseu'
results_file = '/home/linxy29/holab/iPSC/veloAE/' + sample + '_seurate.h5ad'  # the file that will store the analysis results

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [ ]:
# get meta-data information
path = "/home/linxy29/holab/iPSC/veloAE/"
cellID_obs = pd.read_csv(path + sample + '_cellID_obs.csv')
print(cellID_obs.head())
print(len(cellID_obs))
umap_cord = pd.read_csv(path + sample + '_cell_embeddings.csv')
print(umap_cord)
cell_clusters = pd.read_csv(path + sample + '_clusters.csv')

                       x
0  EU_AAACCCAAGCCGATCC-1
1  EU_AAACCCAAGTACAGCG-1
2  EU_AAACCCAGTGCTGATT-1
3  EU_AAACCCAGTTTGGCTA-1
4  EU_AAACCCATCACTTGGA-1
19206
                   Unnamed: 0    UMAP_1    UMAP_2
0       EU_AAACCCAAGCCGATCC-1 -5.482503  6.744282
1       EU_AAACCCAAGTACAGCG-1  6.984378 -2.456347
2       EU_AAACCCAGTGCTGATT-1 -4.731649  0.625126
3       EU_AAACCCAGTTTGGCTA-1 -5.511836 -1.759254
4       EU_AAACCCATCACTTGGA-1 -4.527056  3.496476
...                       ...       ...       ...
19201  CFS_TTTGTTGCATGCTGCG-1 -5.946149  4.723394
19202  CFS_TTTGTTGGTACAGTTC-1 -4.662707 -1.147670
19203  CFS_TTTGTTGTCCGCAACG-1 -3.891447  6.155768
19204  CFS_TTTGTTGTCCGTGGCA-1  4.999151 -5.996508
19205  CFS_TTTGTTGTCGGTAACT-1  6.555681 -2.408121

[19206 rows x 3 columns]


In [ ]:
filenameL = []
os.chdir("/home/linxy29/holab/iPSC/veloAE/day7_velocyto/")
for file in glob.glob("*.loom"):
    filenameL.append(file)
print(filenameL)

['190PBMCiPSC_D7NT_sample_alignments_LNJIY.loom', '190PBMCiPSC_D7NOTO_sample_alignments_OV87U.loom', '190PBMCiPSC_D7NOTO_drug_sample_alignments_P1WVY.loom', 'ThermoFiPSC_D7NT_sample_alignments_Z46BN.loom', 'ThermoFiPSC_D7NOTO_sample_alignments_BT23H.loom', 'ThermoFiPSC_D7NOTO_drug_sample_alignments_PUHUH.loom']


In [ ]:
eudata =  scv.read("/home/linxy29/holab/iPSC/veloAE/day7_velocyto/" + filenameL[0], cache=True)
eudata.var_names_make_unique()

... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-day7_velocyto-190PBMCiPSC_D7NT_sample_alignments_LNJIY.h5ad


In [ ]:
for sample in filenameL[1:]:
    tempdata = scv.read("/home/linxy29/holab/iPSC/veloAE/day7_velocyto/" + sample, cache=True)
    tempdata.var_names_make_unique()
    eudata = eudata.concatenate(tempdata, join='outer', index_unique=None)
print(eudata)
obs_name_temp = eudata.obs_names.str.split(":")
eudata.obs_names = ['EU_' + item[1] + '-1' for item in obs_name_temp]
print(eudata.obs_names)
print(np.unique(eudata.obs_names).size == len(eudata.obs_names))

... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-day7_velocyto-190PBMCiPSC_D7NOTO_sample_alignments_OV87U.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-day7_velocyto-190PBMCiPSC_D7NOTO_drug_sample_alignments_P1WVY.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-day7_velocyto-ThermoFiPSC_D7NT_sample_alignments_Z46BN.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-day7_velocyto-ThermoFiPSC_D7NOTO_sample_alignments_BT23H.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-day7_velocyto-ThermoFiPSC_D7NOTO_drug_sample_alignments_PUHUH.h5ad
AnnData object with n_obs × n_vars = 10656 × 61544
    obs: 'batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['EU_ACCCAAATCACTACTT-1', 'EU_AAGACAATCCTAGCGG-1',
       'EU_AAACGCTAGCGTCTGC-1', 'EU_AAGCGAGGTGTCGATT-1',
       'EU_AAAGTCCAGACCATGG-1', 'EU_AAAGGTAGTAGCACGA-1',

In [ ]:
## add cfs data
cur_sample = 'cfs'
ldata = scv.read('/home/linxy29/holab/iPSC/' + cur_sample + '_cellranger/velocyto/' + cur_sample + '_cellranger.loom', cache=True)
obs_name_rep = ldata.obs_names.str.replace('cfs_cellranger:|x', '') + '-1'
ldata.obs_names = ['CFS_' + item for item in obs_name_rep]
ldata.var_names_make_unique()
print(ldata)
print(ldata.obs_names)
print(np.unique(ldata.obs_names).size == len(ldata.obs_names))

... reading from cache file cache/home-linxy29-holab-iPSC-cfs_cellranger-velocyto-cfs_cellranger.h5ad
AnnData object with n_obs × n_vars = 10275 × 36601
    obs: 'Clusters', '_X', '_Y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['CFS_AAAGGTACACTCTAGA-1', 'CFS_AAACGAACAAACTCTG-1',
       'CFS_AAACCCAGTAATCAGA-1', 'CFS_AAACGCTCAGAGTCAG-1',
       'CFS_AAAGGATAGAGGGTCT-1', 'CFS_AAAGGATAGTAATCCC-1',
       'CFS_AAACGCTCAGCATCTA-1', 'CFS_AAAGGATCATTGCAAC-1',
       'CFS_AAACGAAGTCATAGTC-1', 'CFS_AAACCCATCGTGTGAT-1',
       ...
       'CFS_TTTCAGTAGGTCATAA-1', 'CFS_TTTGACTTCTTCGATT-1',
       'CFS_TTTCCTCTCGGCCCAA-1', 'CFS_TTTGATCGTGTTCCAA-1',
       'CFS_TTTGATCGTCCACACG-1', 'CFS_TTTCATGGTAGGTGCA-1',
       'CFS_TTTGATCAGGAATGTT-1', 'CFS_TTTCGATTCACGTAGT-1',
       'CFS_TTTCGATTCCACTTTA-1', 'CFS_TTTGTTGAGTGCAACG-1'],
      dtype='object', length=10275)
True


In [ ]:
ldata = ldata.concatenate(eudata, join='outer', index_unique=None)
print(ldata)

AnnData object with n_obs × n_vars = 20931 × 74559
    obs: 'Clusters', '_X', '_Y', 'batch'
    var: 'Accession-0', 'Chromosome-0', 'End-0', 'Start-0', 'Strand-0', 'Accession-1', 'Chromosome-1', 'End-1', 'Start-1', 'Strand-1'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


## barcodes

In [ ]:
## change obs_names
print(ldata.obs_names)
print(np.unique(ldata.obs_names).size == len(ldata.obs_names))
print(len(ldata.obs_names))

Index(['CFS_AAAGGTACACTCTAGA-1', 'CFS_AAACGAACAAACTCTG-1',
       'CFS_AAACCCAGTAATCAGA-1', 'CFS_AAACGCTCAGAGTCAG-1',
       'CFS_AAAGGATAGAGGGTCT-1', 'CFS_AAAGGATAGTAATCCC-1',
       'CFS_AAACGCTCAGCATCTA-1', 'CFS_AAAGGATCATTGCAAC-1',
       'CFS_AAACGAAGTCATAGTC-1', 'CFS_AAACCCATCGTGTGAT-1',
       ...
       'EU_TTGGGTAGTTTGTGGT-1', 'EU_TTGTGTTCAGCCGGTT-1',
       'EU_TTCTAACAGACCGCCT-1', 'EU_TTTGTTGAGGCACCAA-1',
       'EU_TTCGGTCAGGGATGTC-1', 'EU_TTGTTCAGTTATCCAG-1',
       'EU_TTCTAGTAGTTCTCTT-1', 'EU_TTTGGTTAGGGCAATC-1',
       'EU_TTCCGTGTCCGACAGC-1', 'EU_TTGCGTCAGTTGGACG-1'],
      dtype='object', length=20931)
True
20931


In [ ]:
## change cellID name in metadata
print(np.unique(cellID_obs['x']).size == len(cellID_obs['x']))  ## check whether have duplicate barcodes
print(len(cellID_obs))

True
19206


In [ ]:
filtered_barcode = np.intersect1d(cellID_obs['x'],ldata.obs_names)
len(filtered_barcode)

18708

In [ ]:
## subset filtered_barcode in ldata
filtered_ldata = ldata[filtered_barcode].copy()
print(filtered_ldata)

AnnData object with n_obs × n_vars = 18708 × 74559
    obs: 'Clusters', '_X', '_Y', 'batch'
    var: 'Accession-0', 'Chromosome-0', 'End-0', 'Start-0', 'Strand-0', 'Accession-1', 'Chromosome-1', 'End-1', 'Start-1', 'Strand-1'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


Add UMAP and annotation

In [ ]:
ldata_index = pd.DataFrame(filtered_ldata.obs.index)
ldata_index.columns = ['CellID']
print(ldata_index)

                       CellID
0      CFS_AAACCCAAGACATATG-1
1      CFS_AAACCCAAGATAGCTA-1
2      CFS_AAACCCAAGGCCTGCT-1
3      CFS_AAACCCAAGGTCACAG-1
4      CFS_AAACCCAAGTCAGCCC-1
...                       ...
18703   EU_TTTGTTGCATCTCAAG-1
18704   EU_TTTGTTGGTAGACGTG-1
18705   EU_TTTGTTGGTGAGCGAT-1
18706   EU_TTTGTTGGTTCTCTAT-1
18707   EU_TTTGTTGTCACCTCAC-1

[18708 rows x 1 columns]


In [ ]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'CellID'})
#umap_cord = umap_cord.rename(columns = {'Cell ID':'CellID'})
umap_ordered = ldata_index.merge(umap_cord, on = "CellID")
print(umap_cord)

                       CellID    UMAP_1    UMAP_2
0       EU_AAACCCAAGCCGATCC-1 -5.482503  6.744282
1       EU_AAACCCAAGTACAGCG-1  6.984378 -2.456347
2       EU_AAACCCAGTGCTGATT-1 -4.731649  0.625126
3       EU_AAACCCAGTTTGGCTA-1 -5.511836 -1.759254
4       EU_AAACCCATCACTTGGA-1 -4.527056  3.496476
...                       ...       ...       ...
19201  CFS_TTTGTTGCATGCTGCG-1 -5.946149  4.723394
19202  CFS_TTTGTTGGTACAGTTC-1 -4.662707 -1.147670
19203  CFS_TTTGTTGTCCGCAACG-1 -3.891447  6.155768
19204  CFS_TTTGTTGTCCGTGGCA-1  4.999151 -5.996508
19205  CFS_TTTGTTGTCGGTAACT-1  6.555681 -2.408121

[19206 rows x 3 columns]


In [ ]:
umap_ordered = umap_ordered.iloc[:,1:]
filtered_ldata.obsm['X_umap'] = umap_ordered.values

In [ ]:
cell_clusters = cell_clusters.rename(columns = {'Unnamed: 0':'CellID'})
cell_clusters = ldata_index.merge(cell_clusters, on = "CellID")
print(cell_clusters)

                       CellID                              x
0      CFS_AAACCCAAGACATATG-1    neuromesodermal progenitors
1      CFS_AAACCCAAGATAGCTA-1                  neuroectoderm
2      CFS_AAACCCAAGGCCTGCT-1                      notochord
3      CFS_AAACCCAAGGTCACAG-1  paraxial mesoderm progenitors
4      CFS_AAACCCAAGTCAGCCC-1              axial progenitors
...                       ...                            ...
18703   EU_TTTGTTGCATCTCAAG-1               primitive streak
18704   EU_TTTGTTGGTAGACGTG-1                  neuroectoderm
18705   EU_TTTGTTGGTGAGCGAT-1                        cardiac
18706   EU_TTTGTTGGTTCTCTAT-1  paraxial mesoderm progenitors
18707   EU_TTTGTTGTCACCTCAC-1               primitive streak

[18708 rows x 2 columns]


In [ ]:
cell_clusters = cell_clusters.iloc[:,1:]
filtered_ldata.obs['seurat_clusters'] = cell_clusters.values
print(filtered_ldata)

AnnData object with n_obs × n_vars = 18708 × 74559
    obs: 'Clusters', '_X', '_Y', 'batch', 'seurat_clusters'
    var: 'Accession-0', 'Chromosome-0', 'End-0', 'Start-0', 'Strand-0', 'Accession-1', 'Chromosome-1', 'End-1', 'Start-1', 'Strand-1'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


In [ ]:
filtered_ldata.write(results_file)

: 